# **Imports**

In [11]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geocoder

# **Scraping Data From URL**

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data=requests.get(url).text
soup=BeautifulSoup(data,'html.parser')

#find the first html table in the web page
table = soup.find('table')

**Function to clean the dataframe column**

In [5]:
def cleanData(df,column):
  # replacing the remaining special characters with the ones mentioned
  df[column]=df[column].str.replace(')','')
  df[column]=df[column].str.replace('(','')
  df[column]=df[column].str.replace('/',',')
  return df

**Reading data into a pandas dataframe**

In [6]:
# creating a new dataframe
toronto_neighbourhood=pd.DataFrame(columns=['Postal_Code','Borough','Neighbourhood'])

# iterating the html tags and scaping the required information
for row in table.find_all('td'):
  postal_code=row.find('b').text
  borough_neighbourhood=row.find('span').text

  if ("Not assigned"  not in borough_neighbourhood):
    borough,neighbourhood=borough_neighbourhood.split('(',1)
    toronto_neighbourhood=toronto_neighbourhood.append({'Postal_Code':postal_code,
                                  'Borough':borough,
                                  'Neighbourhood':neighbourhood
                                  },ignore_index=True)

# cleaning the values
toronto_neighbourhood=cleanData(toronto_neighbourhood,'Neighbourhood')
toronto_neighbourhood

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


**Shape of the dataframe**

In [12]:
toronto_neighbourhood.shape 

(103, 3)

**The api was not working so downloaded the given CSV file**

In [ ]:
"""# initialize your variable to None
lat_lng_coords = None

for i in toronto_neighbourhood['Postal_Code'].to_list():
  # loop until you get the coordinates
  while (lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(str(i)))
    lat_lng_coords = g.latlng
    if (lat_lng_coords) is not None:
      latitude = lat_lng_coords[0]
      longitude = lat_lng_coords[1]
      print('Lat: {} Lng: {}'.format(latitude,longitude))"""

**Reading the geospatial coordinates into a csv file**


In [23]:
geospatial_coordinates=pd.read_csv('Geospatial_Coordinates.csv')
geospatial_coordinates.rename(columns={"Postal Code":"Postal_Code"},inplace=True)
geospatial_coordinates

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


**Combining both dataframes based on postal code**

In [26]:
toronto_neighbourhood=pd.merge(toronto_neighbourhood,geospatial_coordinates,on='Postal_Code')
toronto_neighbourhood

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
